In [ ]:
# 출력 파일 생성 파트2 (sample_submission.csv 생성)
# 리랭커 변경후 사용.

In [ ]:
# 1. 데이터 파일 읽기

import pickle
with open('./data/all_data.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
#print(loaded_data) 
print(loaded_data[0])

In [ ]:
# 2-1   Qwen/Qwen3-Reranker-8B 리랭커 사용시 루트

import json
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM
import argparse
import os

def format_instruction(instruction, query, doc):
    if instruction is None:
        instruction = 'Given a web search query, retrieve relevant passages that answer the query'
    output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
        instruction=instruction, query=query, doc=doc
    )
    return output

class QwenReranker:
    def __init__(self, model_name="Qwen/Qwen3-Reranker-8B", use_flash_attention=False, device=None):
        print(f"Qwen Reranker 모델 로딩 중: {model_name}")
        
        # 토크나이저 로딩
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side='left')
        
        # 모델 로딩
        if use_flash_attention:
            # device_map="auto"를 사용하면 모델이 여러 GPU에 자동으로 분산됩니다.
            # 이때는 모델 전체를 특정 device로 다시 옮기지 않습니다.
            self.model = AutoModelForCausalLM.from_pretrained(
                model_name, 
                torch_dtype=torch.float16, 
                attn_implementation="flash_attention_2",
                device_map="auto"
            ).eval()
            # 입력 텐서를 배치할 기본 장치 설정. 보통 cuda:0이 됩니다.
            # model.device를 사용하면 분산 모델의 경우 'cpu'를 반환할 수 있으므로, 명시적으로 설정
            self.device = "cuda" if torch.cuda.is_available() else "cpu"
            print(f"모델이 'auto' device_map으로 로딩되어 여러 GPU에 분산되었습니다. 입력은 {self.device}로 이동됩니다.")
        else:
            # Flash Attention을 사용하지 않는 경우, 모델을 지정된 단일 장치로 옮깁니다.
            self.model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16).eval()
            # 디바이스 설정
            if device is None:
                device = "cuda" if torch.cuda.is_available() else "cpu"
            self.device = device
            self.model = self.model.to(self.device) # 모델을 단일 장치로 이동
            print(f"모델이 {self.device}로 로딩되었습니다.")
        
        # 토큰 ID 설정
        self.token_false_id = self.tokenizer.convert_tokens_to_ids("no")
        self.token_true_id = self.tokenizer.convert_tokens_to_ids("yes")
        self.max_length = 8192
        
        # 프리픽스와 서픽스 설정
        prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
        suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
        self.prefix_tokens = self.tokenizer.encode(prefix, add_special_tokens=False)
        self.suffix_tokens = self.tokenizer.encode(suffix, add_special_tokens=False)
        
            
    def process_inputs(self, pairs):
        """입력 텍스트들을 토크나이즈하고 처리"""
        # attention_mask도 함께 반환하도록 설정
        inputs = self.tokenizer(
            pairs, 
            padding=False, # initial padding=False
            truncation='longest_first',
            return_attention_mask=True, # attention_mask를 반환하도록 설정
            max_length=self.max_length - len(self.prefix_tokens) - len(self.suffix_tokens)
        )
        
        # 프리픽스와 서픽스 추가 및 attention_mask 업데이트
        for i in range(len(inputs['input_ids'])):
            original_input_ids = inputs['input_ids'][i]
            original_attention_mask = inputs['attention_mask'][i]

            inputs['input_ids'][i] = self.prefix_tokens + original_input_ids + self.suffix_tokens
            # 새로 추가된 토큰에 맞춰 attention_mask 갱신
            inputs['attention_mask'][i] = [1] * len(self.prefix_tokens) + original_attention_mask + [1] * len(self.suffix_tokens)
        
        # 최종 패딩 (input_ids와 attention_mask 모두 처리)
        inputs = self.tokenizer.pad(inputs, padding=True, return_tensors="pt", max_length=self.max_length)
        
        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        return inputs

    def compute_logits(self, inputs):
        with torch.no_grad():
            batch_scores = self.model(**inputs).logits[:, -1, :]
            true_vector = batch_scores[:, self.token_true_id]
            false_vector = batch_scores[:, self.token_false_id]
            batch_scores = torch.stack([false_vector, true_vector], dim=1)
            batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
            scores = batch_scores[:, 1].exp().tolist()
        return scores
    
    def rerank_documents(self, query, documents, batch_size=32, task=None):
        if task is None:
            task = 'Given a web search query, retrieve relevant passages that answer the query'
        
        # 모든 쿼리-문서 쌍을 형식화
        pairs = [format_instruction(task, query, doc) for doc in documents]
        
        all_scores = []
        
        # 배치 단위로 처리
        for i in tqdm(range(0, len(pairs), batch_size), desc="문서 재순위 매기는 중"):
            batch_pairs = pairs[i:i + batch_size]
            inputs = self.process_inputs(batch_pairs)
            scores = self.compute_logits(inputs)
            all_scores.extend(scores)
        
        return all_scores

In [ ]:
# 2-2   Qwen/Qwen3-Reranker-8B 리랭커 사용시 루트

# 리스트 불러오기
import pickle
import json

def reranking(data):

    # Reranker 초기화
    reranker = QwenReranker(
        model_name='Qwen/Qwen3-Reranker-8B',
        use_flash_attention=False
    )

    message = data["message"]
    references = data["references"]
    documents = [doc['content'] for doc in references]
    
    rerank_scores = reranker.rerank_documents(message, documents, batch_size=1)

    print("*"*50)
    print(references)
    print(rerank_scores)

    results = list(zip(references, rerank_scores))
    results.sort(key=lambda x: x[1], reverse=True) 
    final_result = results[:3]
    print(f"final_result={final_result}")

    data['topk'] = [ doc['docid'] for doc, socre in final_result]
    data["references"] = [ {"score": float(score), "content": doc['content']} for doc, score in final_result ]

    return data



with open("./data/sample_submission.csv", "w") as of:
        idx = 0
        for data in loaded_data:

            #if idx == 10: break;

            print(f"data={data}")
            if len(data["topk"]) > 0:
                response = reranking(data)

            print(f'response: {response}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": response["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            print(output)
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1     

In [ ]:
# 3   기타 리랭커 사용시 루트

# 리스트 불러오기
import pickle
import json

from FlagEmbedding import FlagLLMReranker, FlagReranker 

def reranking(data):

    reranker = FlagReranker('BAAI/bge-reranker-v2-m3')
    #reranker = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_fp16=True)
    #reranker = FlagLLMReranker('Qwen/Qwen3-Reranker-8B', use_fp16=True, batch_size=1, max_length=4096)

    message = data["message"]
    references = data["references"]
    pairs = [[message, doc['content']] for doc in references]
    rerank_scores = reranker.compute_score(pairs)

    print("*"*50)
    print(pairs)
    print(references)
    print(rerank_scores)


    results = list(zip(references, rerank_scores))
    results.sort(key=lambda x: x[1], reverse=True) 
    final_result = results[:3]
    print(f"final_result={final_result}")

    data['topk'] = [ doc['docid'] for doc, socre in final_result]
    data["references"] = [ {"score": float(score), "content": doc['content']} for doc, score in final_result ]

    return data


with open("./data/sample_submission.csv", "w") as of:
        idx = 0
        for data in loaded_data:
            print(f"data={data}")
            if len(data["topk"]) > 0:
                response = reranking(data)

            print(f'response: {response}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": response["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            print(output)
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1     

In [ ]:
# 4  리랭커 앙상블 사용시 루트

# 리스트 불러오기
import pickle
import json
import numpy as np

from FlagEmbedding import FlagLLMReranker, FlagReranker 

def rank_based_ensemble(score_cross, score_llm, weights=[0.5, 0.5]):
    """랭크를 기반으로 앙상블"""
    
    # 각 스코어를 랭크로 변환 (높은 점수 = 낮은 랭크)
    rank_cross = np.argsort(np.argsort(-np.array(score_cross)))
    rank_llm = np.argsort(np.argsort(-np.array(score_llm)))
    
    # 랭크를 0-1로 정규화
    n = len(rank_cross)
    norm_rank_cross = rank_cross / (n - 1)
    norm_rank_llm = rank_llm / (n - 1)
    
    # 가중 평균
    final_ranks = weights[0] * norm_rank_cross + weights[1] * norm_rank_llm
    
    # 최종 스코어로 변환 (낮은 랭크 = 높은 스코어)
    return 1 - final_ranks

def reranking(data):

    reranker_cross = FlagReranker('BAAI/bge-reranker-v2-m3', batch_size=8)
    reranker_llm = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', batch_size=8)
    

    message = data["message"]
    references = data["references"]
    pairs = [[message, doc['content']] for doc in references]
    score_cross = reranker_cross.compute_score(pairs)
    score_llm = reranker_llm.compute_score(pairs)

    rerank_scores = rank_based_ensemble(score_cross, score_llm)

    print("*"*50)
    print(pairs)
    print(references)
    print(rerank_scores)


    results = list(zip(references, rerank_scores))
    results.sort(key=lambda x: x[1], reverse=True) 
    final_result = results[:3]
    print(f"final_result={final_result}")

    data['topk'] = [ doc['docid'] for doc, socre in final_result]
    data["references"] = [ {"score": float(score), "content": doc['content']} for doc, score in final_result ]

    return data


with open("./data/sample_submission.csv", "w") as of:
        idx = 0
        for data in loaded_data:
            print(f"data={data}")
            if len(data["topk"]) > 0:
                response = reranking(data)

            print(f'response: {response}\n')

            # 대회 score 계산은 topk 정보를 사용, answer 정보는 LLM을 통한 자동평가시 활용
            output = {"eval_id": response["eval_id"], "standalone_query": response["standalone_query"], "topk": response["topk"], "answer": response["answer"], "references": response["references"]}
            print(output)
            of.write(f'{json.dumps(output, ensure_ascii=False)}\n')
            idx += 1     